In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-insincere-questions-classification/sample_submission.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip
/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/quora-insincere-questions-classification/test.csv


In [2]:
data = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/train.csv", nrows=500)
data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [34]:
data['target'].value_counts()

0    472
1     28
Name: target, dtype: int64

In [3]:
!pip install transformers accelerate bitsandbytes peft

In [4]:
! pip install datasets

In [5]:
from datasets import Dataset

train_ds = Dataset.from_pandas(data[:400])
test_ds = Dataset.from_pandas(data[400:])

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small",load_in_8bit=True)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [8]:
from peft import prepare_model_for_int8_training

model = prepare_model_for_int8_training(model)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [9]:
from peft import LoraConfig, get_peft_model

loraconfig =LoraConfig(r=16, lora_alpha=32, target_modules=['q','v'], lora_dropout=0.05, bias="none", task_type="SEQ_2_SEQ_LM")

model = get_peft_model(model,loraconfig)

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["question_text"], padding="max_length", truncation=True)


#train_tokenized_datasets = train_ds.map(tokenize_function, batched=True)
#test_tokenized_datasets = test_ds.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
data.columns.tolist()

['qid', 'question_text', 'target']

In [45]:
labels_dict = {1:'insincere',0:'sincere'}

In [57]:
type(train_ds)

datasets.arrow_dataset.Dataset

In [58]:
# data preprocessing
text_column = "question_text"
label_column = "target"
max_length = 128


def preprocess_function(examples):
    inputs = examples[text_column]
    targets = [str(x) for x in examples[label_column]] 
    # Don't know why but converting 1 to '1' and then encoding gives better results than converting 1 to 'insincere'!
    # May be because the sampling is poor and just 500 records are selected at random
    #print(type(targets[2]))
    #print(type(inputs[2]))
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="np")
    #print(model_inputs)
    labels = tokenizer(targets, max_length=3, padding="max_length", truncation=True, return_tensors="np")
    #print(labels)
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels#.tolist()
    #print(model_inputs["labels"])
    #labels1 = np.array(examples["target"]).reshape(-1,1)
    #model_inputs["labels"] = labels1.tolist()
    return model_inputs


train_dataset = train_ds.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=data.columns.tolist(),
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

test_dataset = test_ds.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=data.columns.tolist(),
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)


Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
"""
train_tokenized_datasets = train_tokenized_datasets.remove_columns(['qid','question_text'])
train_tokenized_datasets = train_tokenized_datasets.rename_column('target','labels')
train_tokenized_datasets = train_tokenized_datasets.with_format("torch")

test_tokenized_datasets = test_tokenized_datasets.remove_columns(['qid','question_text'])
test_tokenized_datasets = test_tokenized_datasets.rename_column('target','labels')
test_tokenized_datasets = test_tokenized_datasets.with_format("torch")
"""

'\ntrain_tokenized_datasets = train_tokenized_datasets.remove_columns([\'qid\',\'question_text\'])\ntrain_tokenized_datasets = train_tokenized_datasets.rename_column(\'target\',\'labels\')\ntrain_tokenized_datasets = train_tokenized_datasets.with_format("torch")\n\ntest_tokenized_datasets = test_tokenized_datasets.remove_columns([\'qid\',\'question_text\'])\ntest_tokenized_datasets = test_tokenized_datasets.rename_column(\'target\',\'labels\')\ntest_tokenized_datasets = test_tokenized_datasets.with_format("torch")\n'

In [15]:
#train_tokenized_datasets

Dataset({
    features: ['qid', 'question_text', 'target', 'input_ids', 'attention_mask'],
    num_rows: 400
})

In [55]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [61]:
train_dataset[100]['labels']

[3, 632, 1]

In [41]:
model.print_trainable_parameters()

trainable params: 688,128 || all params: 77,649,280 || trainable%: 0.8862001038515747


In [19]:
from transformers import DataCollatorForSeq2Seq, DataCollatorWithPadding

#data_collator = DataCollatorForSeq2Seq(tokenizer,model=model,pad_to_multiple_of=8,label_pad_token_id=-100)

data_collator = DataCollatorWithPadding(tokenizer,max_length =128,return_tensors ="pt")

In [59]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    "temp",
    evaluation_strategy="epoch",
    learning_rate=1e-3,
    gradient_accumulation_steps=1,
    auto_find_batch_size=True,
    num_train_epochs=1,
    save_steps=100,
    save_total_limit=8,
    report_to="none"
)
trainer = Trainer(
    model=model,
    args=training_args,
    #data_collator = data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
model.config.use_cache = False

In [60]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.083813


TrainOutput(global_step=25, training_loss=0.3222392654418945, metrics={'train_runtime': 11.5881, 'train_samples_per_second': 34.518, 'train_steps_per_second': 2.157, 'total_flos': 18800443392000.0, 'train_loss': 0.3222392654418945, 'epoch': 1.0})